In [8]:
import pandas as pd 

df= pd.read_excel("base_conocimiento_colombia.xlsx")
df=df.head(5)
df

,id_documento,filename,title,text,tags,keyword
0,1,departamento_1.pdf,Dato curioso sobre Amazonas,El Amazonas colombiano alberga parte del mayor...,"Colombia,dato_curioso,departamento",Amazonas
1,2,departamento_2.pdf,Dato curioso sobre Antioquia,La piedra del Peñol en Guatapé es una de las m...,"Colombia,dato_curioso,departamento",Antioquia
2,3,departamento_3.pdf,Dato curioso sobre Arauca,Arauca es famosa por su producción de ganado c...,"Colombia,dato_curioso,departamento",Arauca
3,4,departamento_4.pdf,Dato curioso sobre Atlántico,En Barranquilla se celebra uno de los carnaval...,"Colombia,dato_curioso,departamento",Atlántico
4,5,departamento_5.pdf,Dato curioso sobre Bolívar,"Cartagena, en Bolívar, fue uno de los puertos ...","Colombia,dato_curioso,departamento",Bolívar


In [9]:
# se inicializa vertex - toma credenciales default 

from google.cloud import aiplatform

# Reemplaza con tu proyecto y ubicación si es necesario
aiplatform.init(
    project="mestria-puj-s2",       
    location="us-central1"             
)


In [10]:
# carga del modelo de embeddings 

from vertexai.language_models import TextEmbeddingModel

# Cargar el modelo
model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")


In [11]:
# Generar embeddings para la columna 'text'
embeddings = []

for text in df['text']:
    response = model.get_embeddings([text])
    embedding = response[0].values
    embeddings.append(embedding)

# Añadir columna al DataFrame
df["embedding"] = embeddings
df

,id_documento,filename,title,text,tags,keyword,embedding
0,1,departamento_1.pdf,Dato curioso sobre Amazonas,El Amazonas colombiano alberga parte del mayor...,"Colombia,dato_curioso,departamento",Amazonas,"[0.005636319983750582, -0.003553522750735283, ..."
1,2,departamento_2.pdf,Dato curioso sobre Antioquia,La piedra del Peñol en Guatapé es una de las m...,"Colombia,dato_curioso,departamento",Antioquia,"[0.00701145688071847, -0.0027742551174014807, ..."
2,3,departamento_3.pdf,Dato curioso sobre Arauca,Arauca es famosa por su producción de ganado c...,"Colombia,dato_curioso,departamento",Arauca,"[0.009727761149406433, -0.011515113525092602, ..."
3,4,departamento_4.pdf,Dato curioso sobre Atlántico,En Barranquilla se celebra uno de los carnaval...,"Colombia,dato_curioso,departamento",Atlántico,"[-0.03834875300526619, 0.011282055638730526, 0..."
4,5,departamento_5.pdf,Dato curioso sobre Bolívar,"Cartagena, en Bolívar, fue uno de los puertos ...","Colombia,dato_curioso,departamento",Bolívar,"[-0.020088303834199905, -0.004341783933341503,..."


In [15]:
with open("bulk_embeddings.json", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        # Línea de acción
        f.write(json.dumps({ "index": {} }) + "\n")

        # Documento en formato plano (sin _index ni _source)
        doc = {
            "embedding": row["embedding"],
            "text": row["text"],
            "metadata": {
                "filename": row["filename"],
                "keyword": row["keyword"],
                "tags": row["tags"],
                "id_documento": int(row["id_documento"]),
                "title": row["title"]
            }
        }
        f.write(json.dumps(doc) + "\n")